In [1]:
#1
import numpy as np
%pylab 
#style.use('ggplot')
#rcParams['figure.figsize'] = 12,8
import numba as nb
from numba import jit

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import copy
import networkx as nx
from copy import copy, deepcopy



import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#import connectivity as ct

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
from catboost import CatBoostClassifier, Pool, cv

In [5]:
#Customary libraries
import connectivities as cn #Different network connectivities
import physiological_neurons as ph #Dynamics of leaky integrate ad fire neurons
import parameters as pr #Parameters of the simulation 

def data_with_saving(data_size, first_powers, second_powers):
    
    par = pr.my_params()
    N = int(par['n_neurons']) #total number of neurons
    n_activated = int(par['Number of laser activated neurons'])
    n_timesteps = int(par['n_timesteps'])
    noise_fraction = 0.0 #fraction of noicy neurons

    noise_vector = ph.create_noise_vector(N, noise_fraction)

    my_dict = {}
    my_dict["Answer"] = []
    
    
    idm = np.identity(N)
    for i in range(first_powers * 2 * second_powers):
        my_dict[i] = []
        
    for i in range(data_size):
        
        M, edges, W_matrix, syn_del = cn.generate_network(par)
        
        
        

        set_activated = cn.active_set(N, n_activated) #activated neurons are chosen
    
        input_matrix = ph.prepare_input2(par, set_activated)
        
        X = np.zeros(N)
        
        if (i%10 == 0):
            print(i)
       

        
        W_mat, syn_mat = cn.lists_to_matrices_2(N, edges, W_matrix, syn_del)
        A,B, t = ph.full_process( edges, syn_del, input_matrix, W_matrix,  par, noise_vector)
        if (t < n_timesteps - np.max(syn_del) - 1):
            b = 1
        else:
            b = 0

        
        
        
        for j in range(n_activated):
            a = int(set_activated[j])
            X[a] = 1
        V1 = np.matmul(W_mat,X)
        for s1 in range(first_powers):
            V2 = cn.order_n_vec(V1, s1 + 1, N)
            for s2 in range(2):
                if (s2==0):
                    V3 = np.matmul(idm, V2)
                else:
                    V3 = np.matmul(W_mat, V2)
                for s3 in range(second_powers):
                    V4 = cn.order_n_vec(V3, s3 + 1, N)
                    numb = s3 + s2*second_powers + s1 * 2 * second_powers
                    totres = np.sum(V4)
                    my_dict[numb] = [totres]
        
        
        
        my_dict["Answer"] = b
        df = pd.DataFrame(my_dict)
        df.to_csv('data4.csv', mode='a')
        
    
           

In [6]:
data_size = 100
first_powers = 4
second_powers = 3
data_with_saving(data_size,first_powers, second_powers)  
print("Data obtained")



0
10
20
30
40
50
60
70
80
90
Data obtained


In [ ]:

    
def multiple_splitting(X, y, n_split, split_fraction):
    ac_score = np.zeros(n_split)
    simple_score = np.zeros(n_split)
    categorical_features_indices = np.where(X.dtypes != np.float)[0]
    for i in range(n_split):
        X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size= split_fraction)
        train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
        validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)
        num_iter = 1000
        alpha = 0.2
        
        params = {
            'iterations': 500,
            'learning_rate': 0.1,
            'eval_metric': 'Accuracy',
    #'random_seed': 42,
            'logging_level': 'Silent',
            'use_best_model': False
        }

        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=validate_pool)

        best_model_params = params.copy()
        best_model_params.update({
            'use_best_model': True
        })
     

        ac_score[i] = accuracy_score(y_validation, model.predict(X_validation))
        
    return(ac_score)


def prepare_input(df4, cols):
    X = df4
    all_col = df4.columns
    for c in all_col:
        if (c in cols)==False:
            X = X.drop(c, axis=1)
            
    return X       
        

In [ ]:
df4 = pd.read_csv('data4.csv')

X = prepare_input(df4, ['0'])

y = df4.Answer
n_split = 5
split_fraction = 0.75

multiple_splitting(X, y, n_split, split_fraction)